In [2]:
#필요 라이브러리 로드
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

sns.set(style='whitegrid')
pd.set_option('display.max_rows',500)

In [3]:
#일별 예매데이터와 기상데이터, 코로나데이터, 지하철데이터, 주식시장데이터 결합
#일별 예매건수데이터 로드
df_groupby_day = pd.read_csv('F:\\drive\\KOPIS\\groupby_reservation.csv',encoding='cp949')

In [15]:
#일별 기상데이터로드
#일별 전국 기온
df_temperature = pd.read_csv(
    'F:\\drive\\WebWorkPlace2021\\jupyter\\code\\kopis_data\\temperature_2020_전국.csv'
                             ,encoding='cp949')

#일별 전국 습도
df_humidity = pd.read_csv(
    'F:\\drive\\WebWorkPlace2021\\jupyter\\code\\kopis_data\\humidity_2020_전국.csv'
                             ,encoding='cp949')

#일별 전국 강수량
df_rain = pd.read_csv(
    'F:\\drive\\WebWorkPlace2021\\jupyter\\code\\kopis_data\\rain_2020_전국.csv'
                             ,encoding='cp949')


In [18]:
#간략히 살펴보기
print(df_temperature.info())
print(df_humidity.info())
print(df_rain.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   날짜       366 non-null    object 
 1   평균기온(℃)  366 non-null    float64
 2   최저기온(℃)  366 non-null    float64
 3   최고기온(℃)  366 non-null    float64
dtypes: float64(3), object(1)
memory usage: 11.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      366 non-null    object
 1   평균습도    366 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   날짜       366 non-null    object 
 1   강수량(mm)  366 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.8+ KB

In [20]:
#일별 코로나확진자수데이터 로드
covid19 = pd.read_csv('코로나19일별확진자수.csv', encoding='cp949')

In [23]:
covid19

,등록일시,확진자
0,2020/01/01,0
1,2020/01/02,0
2,2020/01/03,0
3,2020/01/04,0
4,2020/01/05,0
5,2020/01/06,0
6,2020/01/07,0
7,2020/01/08,0
8,2020/01/09,0
9,2020/01/10,0


In [180]:
#일별 지하철데이터 로드
df_subway = pd.read_csv(
    'F:\\drive\\WebWorkPlace2021\\jupyter\\code\\kopis_data\\CARD_SUBWAY_MONTH.csv'
                             ,encoding='cp949')

In [182]:
#지하철 데이터 정제하는 함수 선언
def subway_refiner(df):
    df = df.drop('등록일자',axis=1, inplace=False)
    df['날짜']=df['사용일자']
    df['날짜']=df['날짜'].astype(str)
    df['날짜']=df['날짜'].str[:4]+"/"+df['날짜'].str[4:6]+"/"+\
    df['날짜'].str[6:8]
    df = df.drop('사용일자',axis=1, inplace=False)
    return df

In [183]:
df_subway = subway_refiner(df_subway)

In [164]:
#날짜별로 승차총승객수와 하차총승객수를 더해서 반으로 나누는 함수 선언
def subway_aggregater(df):
    df_subway_refined = df.groupby('날짜').sum()
    df_subway_refined['이용자수']=(df_subway_refined['승차총승객수']+\
    df_subway_refined['하차총승객수'])/2
    df_subway_refined.drop(['승차총승객수','하차총승객수'],axis=1,inplace=True)
    return df_subway_refined

In [185]:
df_subway_aggregated = subway_aggregater(df_subway)

In [186]:
df_subway_aggregated

,이용자수
날짜,
2020/01/01,3466694.5
2020/01/02,7679034.0
2020/01/03,8211982.0
2020/01/04,6027700.5
2020/01/05,4412501.0
2020/01/06,7806476.0
2020/01/07,7611871.0
2020/01/08,7982411.0
2020/01/09,8107989.0


In [4]:
#주식시장 일별 데이터 로드
df_stock = pd.read_csv(
    'F:\\drive\\WebWorkPlace2021\\jupyter\\code\\kopis_data\\201901_202106_주식시장.csv'
                             ,encoding='cp949')

In [3]:
#주식시장 일별 데이터 로드 추가
df_stock = pd.read_csv(
    'F:\\drive\\WebWorkPlace2021\\jupyter\\code\\kopis_data\\주식시장_일별_202107_202108.csv'
                             ,encoding='cp949')

In [4]:
df_date = pd.read_csv(
    'F:\\drive\\WebWorkPlace2021\\jupyter\\code\\kopis_data\\날짜.csv'
                             ,encoding='cp949')

In [5]:
df_stock

,계정별,KOSPI지수 (1980.01.04=100),"거래량(주식시장, 잠정치) (만주)",KOSDAQ지수 (1996.07.01=1000),"거래량(만주 : 코스닥시장, 잠정치) (만주)"
0,2021. 07. 01,3282.06,113665,1035.64,168683
1,2021. 07. 02,3281.78,100670,1038.18,168541
2,2021. 07. 05,3293.21,83928,1047.33,174306
3,2021. 07. 06,3305.21,101782,1044.96,196414
4,2021. 07. 07,3285.34,111563,1047.36,169298
5,2021. 07. 08,3252.68,149605,1034.48,186943
6,2021. 07. 09,3217.95,129242,1028.93,218864
7,2021. 07. 12,3246.47,111875,1034.64,171658
8,2021. 07. 13,3271.38,113211,1043.31,170684
9,2021. 07. 14,3264.81,89595,1044.98,171436


In [6]:
df_date

,날짜
0,2019/01/01
1,2019/01/02
2,2019/01/03
3,2019/01/04
4,2019/01/05
...,...
969,2021/08/27
970,2021/08/28
971,2021/08/29
972,2021/08/30


In [7]:
#날짜컬럼 형식 통일
df_stock['날짜']=df_stock['계정별'].str.split(". ").str[0]+"/"+\
df_stock['계정별'].str.split(". ").str[1]+\
"/"+df_stock['계정별'].str.split(". ").str[2]

In [8]:
df_stock.drop('계정별', axis=1, inplace=True)

In [9]:
df_stock.tail(10)

,KOSPI지수 (1980.01.04=100),"거래량(주식시장, 잠정치) (만주)",KOSDAQ지수 (1996.07.01=1000),"거래량(만주 : 코스닥시장, 잠정치) (만주)",날짜
33,3158.93,61385,1021.08,109540,2021/08/18
34,3097.83,63671,991.15,140186,2021/08/19
35,3060.51,65998,967.90,126610,2021/08/20
36,3090.21,56416,993.18,99351,2021/08/23
37,3138.30,57784,1013.18,132608,2021/08/24
38,3146.81,55766,1017.78,122584,2021/08/25
39,3128.53,71878,1020.44,108165,2021/08/26
40,3133.90,51197,1023.51,129891,2021/08/27
41,3144.19,59529,1031.84,117812,2021/08/30
42,3199.27,64730,1038.33,124347,2021/08/31


In [10]:
#데이터프레임 병합
df = pd.merge(left = df_stock , right = df_date, how = "outer", on = "날짜")

In [13]:
#데이터프레임 날짜 컬럼 기준 오름차순 정렬
df = df.sort_values(by='날짜')

In [15]:
df.to_csv("F:\\drive\\WebWorkPlace2021\\jupyter\\code\\kopis_data\\202107_202108_주식_refined.csv")

In [192]:
#코로나 일별 확진자수 데이터로드 및 전처리
df_covid = pd.read_csv(
    'F:\\drive\\WebWorkPlace2021\\jupyter\\code\\kopis_data\\코로나19일별확진자수.csv'
                             ,encoding='cp949')

In [39]:
print(df_groupby_day.head(1))
print(df_temperature.head(1))
print(df_humidity.head(1))
print(df_rain.head(1))
print(df_stock.head(1))

   예매일시_month  예매일시_day     건수          날짜
0           1         1  51947  2020/01/01
           날짜  평균기온(℃)  최저기온(℃)  최고기온(℃)
0  2020/01/01     -0.9     -6.5      4.1
           날짜  평균습도
0  2020/01/01    63
           날짜  강수량(mm)
0  2020/01/01      0.0
   KOSPI지수  KOSDAQ지수  거래량(KOSPI)  거래량(KOSDAQ) (만주)          날짜
0  2175.17    674.02       48861             78373  2020-01-02


In [187]:
#날짜인덱스 기준으로 병합
temp=pd.merge(left = df_groupby_day , right = df_temperature, how = "inner", on = "날짜")
temp=pd.merge(left = temp , right = df_humidity, how = "inner", on = "날짜")
temp=pd.merge(left = temp , right = df_rain, how = "inner", on = "날짜")
temp=pd.merge(left = temp , right = df_stock, how = "outer", on = "날짜")
temp=pd.merge(left = temp , right = df_subway_aggregated, how = "outer", on = "날짜")
temp=pd.merge(left = temp , right = df_covid, how = "outer", on = "날짜")

In [11]:
temp = pd.merge(left = df_stock , right = df_date, how = "outer", on = "날짜")

In [15]:
temp = temp.sort_values(by='날짜')

In [16]:
temp.to_csv('F:\\drive\\WebWorkPlace2021\\jupyter\\code\\kopis_data\\201901_202106_주식_refined.csv')

In [17]:
#코스피지수, 코스닥지수 이전 날짜로 fillna하기
temp

,KOSPI지수,KOSPI거래량,KOSDAQ지수,KOSDAQ거래량,날짜
617,NaN,NaN,NaN,NaN,2019/01/01
0,2010.00,32176.0,669.37,53945.0,2019/01/02
1,1993.70,42798.0,657.02,65633.0,2019/01/03
2,2010.25,40899.0,664.49,55446.0,2019/01/04
618,NaN,NaN,NaN,NaN,2019/01/05
...,...,...,...,...,...
910,NaN,NaN,NaN,NaN,2021/06/26
911,NaN,NaN,NaN,NaN,2021/06/27
614,3301.89,140494.0,1017.91,158750.0,2021/06/28
615,3286.68,118194.0,1022.52,206194.0,2021/06/29
